In [1]:
!wget https://www.malware-traffic-analysis.net/2019/09/25/2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap.zip

--2020-02-22 09:14:46--  https://www.malware-traffic-analysis.net/2019/09/25/2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap.zip
Resolving www.malware-traffic-analysis.net (www.malware-traffic-analysis.net)... 166.78.135.34
Connecting to www.malware-traffic-analysis.net (www.malware-traffic-analysis.net)|166.78.135.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13973406 (13M) [application/zip]
Saving to: ‘2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap.zip’

2019-09-25-Trickbot 100%[===================>]  13.33M  6.54MB/s    in 2.0s    

2020-02-22 09:14:48 (6.54 MB/s) - ‘2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap.zip’ saved [13973406/13973406]



In [14]:
!unzip  -P infected "2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap.zip" -d pcap_sample

Archive:  2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap.zip
  inflating: pcap_sample/2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap  


In [15]:
ls pcap_sample

2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap


In [16]:
!pip install scapy

     |████████████████████████████████| 911kB 2.9MB/s 
  Created wheel for scapy: filename=scapy-2.4.3-py2.py3-none-any.whl size=1043873 sha256=8c68b6bd01fa8c1e0d616e43d3ee41872901f148bc72b4f7309168c1d9303a1f
  Stored in directory: /root/.cache/pip/wheels/95/bf/51/905b3e84ec4ca910ce4ae92173c7334623105a265bdd1d9438
Successfully built scapy


In [0]:
from scapy.all import rdpcap

In [0]:
packets = rdpcap('./pcap_sample/2019-09-25-Trickbot-gtag-ono19-infection-traffic.pcap')

In [19]:
len(packets)

16731

In [20]:
packets[0]

<Ether  dst=ff:ff:ff:ff:ff:ff src=00:08:02:1c:47:ae type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=328 id=31 flags= frag=0 ttl=128 proto=17 chksum=0x1619 src=10.9.25.101 dst=255.255.255.255 |<UDP  sport=68 dport=67 len=308 chksum=0x985f |<BOOTP  op=BOOTREQUEST htype=1 hlen=6 hops=0 xid=4285390644 secs=0 flags= ciaddr=10.9.25.101 yiaddr=0.0.0.0 siaddr=0.0.0.0 giaddr=0.0.0.0 chaddr=b'\x00\x08\x02\x1cG\xae\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' sname=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' file=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [0]:
# https://github.com/dgunter/pcaptojson/blob/master/pcap_to_json.py

from base64 import b64encode
from scapy.all import rdpcap
import sys

def save_to_mongo(client,pkt):
    db = client.pcap
    postencodepkt = fix_encoding(pkt)
    try:
        db.packets.insert_one(postencodepkt)
    except:
        e = sys.exc_info()[0]
        print("Error: " + str(e))

def fix_encoding(d,target_encoding='utf-8'):
    '''
    @summary: Fixes the encoding within a dict to make the dict JSON encodable
    @var d: A dictionary to fix encoding within
    @var target_encoding: The encoding to check for
    '''
    out_dict = dict()
    for k, v in d.items():
        if type(v) == dict:
            out_dict[k] = fix_encoding(v)
        else:
            # Check to see if value is unicode
            try:
                #str(v).decode(target_encoding)
                out_dict[k] = v
            except UnicodeError:
                # Base64 encode value to avoid json errors
                try:
                    out_dict[k] = b64encode(v)
                except:
                    pass
    # Other packet fixes
    if "DNS" in out_dict:
        out_dict["DNS"]["qd"] = str(out_dict.get("DNS").get("qd"))
        out_dict["DNS"]["an"] = str(out_dict.get("DNS").get("an"))

    return out_dict



def pcap_to_json(pkt):
    '''
    @summary: Converts a collection of scapy packets to a json encodable object
    @var pkt: The scapy packet collection to encode
    @var json_indent: Controls formatting within the JSON encoded data
    @return: Returns a python list of packets
    '''
    pkts = list()
    
    for x in pkt:
        pkts.append(packet_to_json(x))
        
    return pkts

def packet_to_json(pkt):
    '''
    @summary: Converts a collection of scapy packets to a json encodable object
    @var pkt: The scapy packet collection to encode
    @var json_indent: Controls formatting within the JSON encoded data
    '''    
    retpacket = dict()
    layer_iter = 0
    while pkt.getlayer(layer_iter) is not None:
        # Save the layer fields
        layerfields = pkt.getlayer(layer_iter).fields
        # Save the layer name for future identification
        retpacket[pkt.getlayer(layer_iter).name] = fix_encoding(layerfields)
        layer_iter+=1

    return retpacket

def load_pcap_to_mongodb(client,f):
    pcapdata = rdpcap(f)
    for pkt in pcap_to_json(pcapdata):
        save_to_mongo(client, pkt)

In [40]:
pcap_to_json(packets[0])

[{'BOOTP': {'chaddr': b'\x00\x08\x02\x1cG\xae\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'ciaddr': '10.9.25.101',
   'file': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
   'flags': <Flag 0 ()>,
   'giaddr': '0.0.0.0',
   'hlen': 6,
   'hops': 0,
   'htype': 1,
   'op': 1,
   'options': b'c\x82Sc',
   'secs': 0,
   'siaddr': '0.0.0.0',
   'sname': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [181]:
pcap_to_json(packets[32])

[{'DNS': {'aa': 0,
   'ad': 0,
   'an': <DNSRR  rrname='dns.msftncsi.com.' type=A rclass=IN ttl=5 rdlen=None rdata=131.107.255.255 |>,
   'ancount': 1,
   'ar': <DNSRR  rrname='ns3.msft.net.' type=A rclass=IN ttl=5 rdlen=None rdata=193.221.113.53 |<DNSRR  rrname='ns3.msft.net.' type=AAAA rclass=IN ttl=5 rdlen=None rdata=2620:0:34::53 |<DNSRR  rrname='ns1.msft.net.' type=A rclass=IN ttl=5 rdlen=None rdata=208.84.0.53 |<DNSRR  rrname='ns1.msft.net.' type=AAAA rclass=IN ttl=5 rdlen=None rdata=2620:0:30::53 |<DNSRR  rrname='ns2.msft.net.' type=A rclass=IN ttl=5 rdlen=None rdata=208.84.2.53 |<DNSRR  rrname='ns2.msft.net.' type=AAAA rclass=IN ttl=5 rdlen=None rdata=2620:0:32::53 |<DNSRR  rrname='ns4.msft.net.' type=A rclass=IN ttl=5 rdlen=None rdata=208.76.45.53 |<DNSRR  rrname='ns4.msft.net.' type=AAAA rclass=IN ttl=5 rdlen=None rdata=2620:0:37::53 |>>>>>>>>,
   'arcount': 8,
   'cd': 0,
   'id': 32448,
   'length': None,
   'ns': <DNSRR  rrname='msftncsi.com.' type=NS rclass=IN ttl=5 rdlen

In [169]:
%%time
packets_json_list = [pcap_to_json(p)[0] for p in packets]

CPU times: user 7.3 s, sys: 6.94 ms, total: 7.3 s
Wall time: 7.31 s


In [170]:
for i in range(len(packets_json_list)):
    try:
        packets_json_list[i]['DHCP options'] = {op[0]: op[1] for op in packets_json_list[i]['DHCP options']['options']}
        print(packets_json_list[i]['DHCP options'])
    except Exception:
        pass

{'message-type': 8, 'client_id': b'\x01\x00\x08\x02\x1cG\xae', 'hostname': b'Bachmann-BTO-PC', 'vendor_class_id': b'MSFT 5.0', 'param_req_list': [1, 15, 3, 6, 44, 46, 47, 31, 33, 121, 249, 43, 252], 'e': 'n', 'p': 'a'}
{'message-type': 5, 'server_id': '10.9.25.254', 'subnet_mask': '255.255.255.0', 'domain': b'localdomain', 'router': '10.9.25.1', 'name_server': '10.9.25.1', 'NetBIOS_server': '10.9.25.1', 'e': 'n', 'p': 'a'}


In [0]:
import pandas as pd

In [0]:
df = pd.io.json.json_normalize(pcap_to_json(packets[0])[0], sep='_')

In [55]:
df

,Ethernet_dst,Ethernet_src,Ethernet_type,IP_options,IP_version,IP_ihl,IP_tos,IP_len,IP_id,IP_flags,IP_frag,IP_ttl,IP_proto,IP_chksum,IP_src,IP_dst,UDP_sport,UDP_dport,UDP_len,UDP_chksum,BOOTP_options,BOOTP_file,BOOTP_sname,BOOTP_chaddr,BOOTP_giaddr,BOOTP_siaddr,BOOTP_yiaddr,BOOTP_ciaddr,BOOTP_flags,BOOTP_secs,BOOTP_xid,BOOTP_hops,BOOTP_hlen,BOOTP_htype,BOOTP_op,DHCP options_options
0,ff:ff:ff:ff:ff:ff,00:08:02:1c:47:ae,2048,[],4,5,0,328,31,,0,128,17,5657,10.9.25.101,255.255.255.255,68,67,308,39007,b'c\x82Sc',b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,b'\x00\x08\x02\x1cG\xae\x00\x00\x00\x00\x00\x0...,0.0.0.0,0.0.0.0,0.0.0.0,10.9.25.101,,0,4285390644,0,6,1,1,"[(message-type, 8), (client_id, b'\x01\x00\x08..."


In [151]:
%%time
df_list = [pd.io.json.json_normalize(p_j, sep='_') for p_j in packets_json_list]

CPU times: user 50.4 s, sys: 24.6 ms, total: 50.4 s
Wall time: 50.5 s


In [152]:
len(set([tuple(df.columns) for df in df_list]))

15

In [153]:
%%time
df_packets = pd.concat(df_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


CPU times: user 1min 7s, sys: 115 ms, total: 1min 7s
Wall time: 1min 7s


In [154]:
df_packets

,BOOTP_chaddr,BOOTP_ciaddr,BOOTP_file,BOOTP_flags,BOOTP_giaddr,BOOTP_hlen,BOOTP_hops,BOOTP_htype,BOOTP_op,BOOTP_options,BOOTP_secs,BOOTP_siaddr,BOOTP_sname,BOOTP_xid,BOOTP_yiaddr,DHCP options_options_NetBIOS_server,DHCP options_options_client_id,DHCP options_options_domain,DHCP options_options_e,DHCP options_options_hostname,DHCP options_options_message-type,DHCP options_options_name_server,DHCP options_options_p,DHCP options_options_param_req_list,DHCP options_options_router,DHCP options_options_server_id,DHCP options_options_subnet_mask,DHCP options_options_vendor_class_id,DNS Question Record_qclass,DNS Question Record_qname,DNS Question Record_qtype,DNS Resource Record_rclass,DNS Resource Record_rdata,DNS Resource Record_rdlen,DNS Resource Record_rrname,DNS Resource Record_ttl,DNS Resource Record_type,DNS SOA Resource Record_expire,DNS SOA Resource Record_minimum,DNS SOA Resource Record_mname,...,NBT Datagram Packet_ID,NBT Datagram Packet_Length,NBT Datagram Packet_NULL,NBT Datagram Packet_Offset,NBT Datagram Packet_SUFFIX1,NBT Datagram Packet_SUFFIX2,NBT Datagram Packet_SourceIP,NBT Datagram Packet_SourceName,NBT Datagram Packet_SourcePort,NBT Datagram Packet_Type,NTPHeader_delay,NTPHeader_dispersion,NTPHeader_id,NTPHeader_leap,NTPHeader_mode,NTPHeader_orig,NTPHeader_poll,NTPHeader_precision,NTPHeader_recv,NTPHeader_ref,NTPHeader_ref_id,NTPHeader_sent,NTPHeader_stratum,NTPHeader_version,Raw_load,TCP_ack,TCP_chksum,TCP_dataofs,TCP_dport,TCP_flags,TCP_options,TCP_reserved,TCP_seq,TCP_sport,TCP_urgptr,TCP_window,UDP_chksum,UDP_dport,UDP_len,UDP_sport
0,b'\x00\x08\x02\x1cG\xae\x00\x00\x00\x00\x00\x0...,10.9.25.101,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,,0.0.0.0,6.0,0.0,1.0,1.0,b'c\x82Sc',0.0,0.0.0.0,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,4.285391e+09,0.0.0.0,NaN,b'\x01\x00\x08\x02\x1cG\xae',NaN,n,b'Bachmann-BTO-PC',8.0,NaN,a,"[1, 15, 3, 6, 44, 46, 47, 31, 33, 121, 249, 43...",NaN,NaN,NaN,b'MSFT 5.0',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39007.0,67.0,308.0,68.0
0,b'\x00\x08\x02\x1cG\xae\x00\x00\x00\x00\x00\x0...,10.9.25.101,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,,0.0.0.0,6.0,0.0,1.0,2.0,b'c\x82Sc',0.0,10.9.25.254,b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...,4.285391e+09,0.0.0.0,10.9.25.1,NaN,b'localdomain',n,NaN,5.0,10.9.25.1,a,NaN,10.9.25.1,10.9.25.254,255.255.255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34728.0,68.0,308.0,67.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,b'wpad.localdomain.',1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24034.0,53.0,42.0,60314.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,b'Bachmann-BTO-PC.',255.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51531.0,5355.0,41.0,58876.0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"b'""\x00\xf9\x01\x00\x00\x00\x01\x04\x00\x00\x0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [155]:
list(df_packets.columns)

['BOOTP_chaddr',
 'BOOTP_ciaddr',
 'BOOTP_file',
 'BOOTP_flags',
 'BOOTP_giaddr',
 'BOOTP_hlen',
 'BOOTP_hops',
 'BOOTP_htype',
 'BOOTP_op',
 'BOOTP_options',
 'BOOTP_secs',
 'BOOTP_siaddr',
 'BOOTP_sname',
 'BOOTP_xid',
 'BOOTP_yiaddr',
 'DHCP options_options_NetBIOS_server',
 'DHCP options_options_client_id',
 'DHCP options_options_domain',
 'DHCP options_options_e',
 'DHCP options_options_hostname',
 'DHCP options_options_message-type',
 'DHCP options_options_name_server',
 'DHCP options_options_p',
 'DHCP options_options_param_req_list',
 'DHCP options_options_router',
 'DHCP options_options_server_id',
 'DHCP options_options_subnet_mask',
 'DHCP options_options_vendor_class_id',
 'DNS Question Record_qclass',
 'DNS Question Record_qname',
 'DNS Question Record_qtype',
 'DNS Resource Record_rclass',
 'DNS Resource Record_rdata',
 'DNS Resource Record_rdlen',
 'DNS Resource Record_rrname',
 'DNS Resource Record_ttl',
 'DNS Resource Record_type',
 'DNS SOA Resource Record_expire',
 '

In [156]:
df_packets.IP_src.value_counts()

195.123.220.86     8560
10.9.25.101        5606
187.58.56.26        747
185.98.87.185       558
144.91.69.195       500
200.116.199.10      214
10.9.25.1            77
72.21.81.200         67
195.123.221.104      66
104.124.58.155       45
170.238.117.187      34
37.228.117.247       13
185.250.204.126      13
172.217.1.234        12
176.58.123.25        11
107.172.143.91       11
5.53.125.13          11
195.123.238.36       11
185.90.61.116        10
172.217.12.78        10
195.123.221.178      10
186.183.199.114       9
66.85.156.66          9
172.217.9.131         9
23.229.232.193        7
192.227.232.22        6
64.44.51.88           6
37.228.117.146        6
172.217.12.67         6
37.44.212.216         6
45.148.10.48          6
66.55.71.11           6
46.30.41.229          6
198.105.254.64        6
203.23.128.168        6
198.105.244.64        6
31.184.253.37         6
188.225.57.125        6
51.254.69.244         6
185.222.202.222       6
45.14.49.77           5
194.5.250.84    

In [157]:
df_packets.IP_dst.value_counts()

10.9.25.101        11125
195.123.220.86      3305
187.58.56.26         607
10.9.25.1            308
185.98.87.185        244
10.9.25.255          212
144.91.69.195        203
200.116.199.10       127
224.0.0.252           65
72.21.81.200          63
224.0.0.22            61
195.123.221.104       40
170.238.117.187       40
104.124.58.155        25
172.217.1.234         15
172.217.12.78         12
172.217.9.131         11
195.123.238.36        10
5.53.125.13           10
176.58.123.25         10
192.227.232.22         9
37.228.117.146         9
186.183.199.114        9
64.44.51.88            9
45.148.10.48           9
37.44.212.216          9
46.30.41.229           9
66.85.156.66           9
66.55.71.11            9
198.105.244.64         9
195.123.221.178        9
51.254.69.244          9
188.225.57.125         9
185.90.61.116          9
198.105.254.64         9
31.184.253.37          9
185.222.202.222        9
203.23.128.168         9
37.228.117.247         8
185.250.204.126        8


In [158]:
df_packets.TCP_sport.value_counts()

447.0      8753
49528.0    2456
80.0       1114
449.0       970
49187.0     559
           ... 
49513.0       3
49514.0       3
49471.0       2
49198.0       2
445.0         2
Name: TCP_sport, Length: 154, dtype: int64

In [159]:
df_packets.TCP_dport.value_counts()

49528.0    6557
447.0      3475
49187.0    1330
449.0       743
49195.0     650
           ... 
49538.0       1
49541.0       1
49544.0       1
49486.0       1
49540.0       1
Name: TCP_dport, Length: 154, dtype: int64

In [160]:
df_packets.UDP_sport.value_counts()

137.0      371
53.0        75
138.0       21
53157.0      3
57457.0      3
          ... 
64843.0      1
52568.0      1
52592.0      1
52637.0      1
68.0         1
Name: UDP_sport, Length: 132, dtype: int64

In [161]:
df_packets.UDP_dport.value_counts()

137.0      371
53.0       126
5355.0      65
138.0       21
123.0        2
          ... 
56740.0      1
62904.0      1
67.0         1
52763.0      1
68.0         1
Name: UDP_dport, Length: 83, dtype: int64

In [162]:
df_packets.Raw_load.value_counts()

b'\xc0\x0c\x00 \x00\x01\x00\x04\x93\xe0\x00\x06`\x00\n\t\x19e'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [184]:
df_packets.Raw_load.unique()

array([nan,
       b'"\x00\xf9\x01\x00\x00\x00\x01\x04\x00\x00\x00\xe0\x00\x00\xfc',
       b'\xc0\x0c\x00 \x00\x01\x00\x04\x93\xe0\x00\x06\xe0\x00\n\t\x19e',
       ...,
       b'\x17\x03\x01\x00\xb0\x0e`s\x82A7W\xac"\xc1\x0e\xad\xfaXZ\xc5BA\x903X\xc7\x16\xa6Y1\xafL]\x1d\xe8`S\xc8LK^\xf3k.\x9a\xc7\xf3\xa8k\x00\xf4\xf5P\x8c\xd67\xd2I\xcdc\x02{G\x0c\xf9\xec\x045\xb8a\xb0?\x01jC\\h\x98\x01R\xa7X{vEE\x9d!\x04\xcc?L\xa2\x15Z0nc\x0ez\xae\xbc\x06\xc8;\xdd7B\xc4\xa4\xc8\x07"IL\xaf\x8f\x05\t\x9b8\x19\x01>\'\xcc\xa4\x11\xf9\x02\xec\xcf\x1a\x9cV\xb9\x91\xa8\x16&\xf7\x00\xca(\x99 \xe1\'RBJ\xd3\x13\x1a\x99(\xd9a\xf0\xdd]\xb5/p>\t\xdcV\n\xcf\t.#\x880[\x05\xbc\xf7"',
       b'\x15\x03\x01\x00 g\xe7\xdc \xb3Gl\xd4\xa3\xdfU\xe4Z\xabF\xeb<og\xb4\x01\xd7\xac\x9c\x11\xd6e[@\xe3*\xab',
       b'\x15\x03\x01\x00 ?\x84/\xae\xc8\x08*n\xc8Z\x9d\x15\x92k%\xceS-\x08\xe3\xf5Q\x97\xef\r\xcb\x11\xe0;<\xb7\x87'],
      dtype=object)

In [164]:
df_packets['DHCP options_options_hostname'].value_counts()

b'Bachmann-BTO-PC'    1
Name: DHCP options_options_hostname, dtype: int64

In [183]:
df_packets.DNS_ar

0     NaN
0     NaN
0    None
0     NaN
0     NaN
     ... 
0     NaN
0     NaN
0     NaN
0     NaN
0     NaN
Name: DNS_ar, Length: 16731, dtype: object